In [ ]:
!pip install yfinance
!pip install python-telegram-bot
!pip install nest_asyncio

In [283]:
#Importando Bibliotecas
import yfinance as yf
import pandas as pd
import numpy as np



In [284]:
Petr = yf.Ticker('PETR4.SA')
Petr_dia = Petr.history(period='1d',interval='15m')
Petr_dia = Petr_dia.Close
df_Petr_dia = pd.DataFrame(Petr_dia)
df_Petr_dia.reset_index(inplace=True)


In [285]:
# Pegando ultimo valor negociado
Petr_ultimo_preco = df_Petr_dia.tail(1)


In [ ]:
#Renomeando Colunas
Petr_ultimo_preco.rename(columns={'Datetime':'Data','Close':'Preço_Fechamento'}, inplace=True)

In [287]:
# Histórico de ações PETR4 nos ultimos 30 dias
df = pd.read_csv('PETR4.SA.csv',delimiter=',')
df_Petr4 = df[['Date','Close']]
df_Petr4.rename(columns={'Date':'Data','Close':'Preço_Fechamento'}, inplace=True)


In [ ]:
#Ajustando o tipo da data
df_Petr4.loc[:,('Data')]=pd.to_datetime(df_Petr4.loc[:,('Data')], format='%Y-%m-%d')


In [289]:
#Atualização de datas
df_att = df_Petr4.loc[(df_Petr4['Data']==pd.to_datetime('today').normalize())]
df_Petr4 = df_Petr4.drop(df_att.index)

In [290]:
#Adicionando data atual ao histórico
df_Petr_total = df_Petr4.append(Petr_ultimo_preco)


In [291]:
df_Petr_total['Data']=pd.to_datetime(df_Petr_total['Data'], utc=True).dt.date

In [292]:
#Calculo Macd

RapidaMME = df_Petr_total.Preço_Fechamento.ewm(span=12).mean()
LentaMME = df_Petr_total.Preço_Fechamento.ewm(span=26).mean()

Macd = RapidaMME - LentaMME
sinal = Macd.ewm(span=9).mean()

df_Petr_total['Macd'] = Macd
df_Petr_total['Sinal'] = sinal


In [ ]:
#Ajuster de index e retirada de data

df_Petr_total = df_Petr_total.set_index(pd.DatetimeIndex(df_Petr_total['Data'].values))
df_Petr_total = df_Petr_total.drop('Data',1)


In [ ]:
#Verificação de compra ou venda
df_Petr_total['Flag']=''
df_Petr_total['Preço_de_compra']=np.nan
df_Petr_total['Preço_de_venda']=np.nan

for i in range(1,len(df_Petr_total.Sinal)):
  if df_Petr_total['Macd'][i] > df_Petr_total['Sinal'][i]:
    if df_Petr_total['Flag'][i-1]=='C':
      df_Petr_total['Flag'][i]='C'
    else:
      df_Petr_total['Flag'][i]='C'
      df_Petr_total['Preço_de_compra'][i] = df_Petr_total['Preço_Fechamento'][i]
  
  elif df_Petr_total['Macd'][i]< df_Petr_total['Sinal'][i]:
    if df_Petr_total['Flag'][i-1] == 'V':
      df_Petr_total['Flag'][i]='V'
    else:
      df_Petr_total['Flag'][i]='V'
      df_Petr_total['Preço_de_venda'][i] = df_Petr_total['Preço_Fechamento'][i]

In [297]:

import asyncio
import telegram
import nest_asyncio
nest_asyncio.apply()
my_token = '5759866780:AAHDIVBF1AMyOD67_wlMxknv0oy_cH6ZUJ4'
chat_id = '-863310586'


async def envia_msg(msg,chat_id,token = my_token):
  bot = telegram.Bot(token = token)
  await bot.sendMessage(chat_id = chat_id, text=msg)

hoje = df_Petr_total.Flag[-1]
ontem = df_Petr_total.Flag[-2]
Flag = hoje
Preço_Fechamento = round(df_Petr_total.Preço_Fechamento.tail(1)[-1],2)
msg = f'PTR4 (Petrobras) "{Flag}" preço de fechamento : {Preço_Fechamento}'
asyncio.run(envia_msg(msg,chat_id,my_token))